In [3]:
# NoSQL Setup - PyMongo Establishment
# import packages
from pymongo import MongoClient
from pprint import pprint
import json

In [6]:
# instance of local mongo, chain collection and db names together
mongo = MongoClient(port=27017)
uk_food = mongo['uk_food']
establishments = uk_food['establishments']
# load json data from resources folder
with open('Resources/establishments.json', encoding='utf-8') as json_data:
    establishments_json = json.load(json_data)
# insert establishment json into mongo collection, insert_data var is peripheral
# insert_many
insert_data = establishments.insert_many(establishments_json)

### or you could use the mongoimport terminal and import the data set w/
## mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json


In [10]:
# confirm that our new db exists
print(mongo.list_database_names())

['MET', 'admin', 'class_db', 'config', 'fruits_db', 'garden_db', 'local', 'petsitly_marketing', 'test', 'travel_db', 'uk_food']


In [11]:
# confirm db and collection by selecting a random document
pprint(uk_food.establishments.find_one())

{'AddressLine1': 'The Bay',
 'AddressLine2': 'St Margarets Bay',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Refreshment Kiosk',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'ChangesByServerID': 0,
 'Distance': 4587.347174863443,
 'FHRSID': 254719,
 'LocalAuthorityBusinessID': 'PI/000069980',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DY',
 'RatingDate': '2022-03-24T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('650547dd971c31fd40d1d24d'),
 'geocode': {'latitude': '51.152225', 'longitude': '1.387974'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254719',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extractDate': '0001-01-01T0

In [15]:
# update database with an exciting new halal restaurant in Greenwich
# called Penang flavours, no ratings
# before I add it tho, I'm going to aggregate BusinessType to determine the naming convention
query = [{'$group': {'_id': "$BusinessType", 'count': { '$sum': 1 }}}]
results = list(establishments.aggregate(query))
pprint(results)

[{'_id': 'Hotel/bed & breakfast/guest house', 'count': 867},
 {'_id': 'Other catering premises', 'count': 2363},
 {'_id': 'Distributors/Transporters', 'count': 215},
 {'_id': 'Farmers/growers', 'count': 93},
 {'_id': 'Restaurant/Cafe/Canteen', 'count': 10095},
 {'_id': 'Takeaway/sandwich shop', 'count': 4729},
 {'_id': 'Retailers - supermarkets/hypermarkets', 'count': 1100},
 {'_id': 'Importers/Exporters', 'count': 35},
 {'_id': 'Pub/bar/nightclub', 'count': 3913},
 {'_id': 'School/college/university', 'count': 2902},
 {'_id': 'Caring Premises', 'count': 3724},
 {'_id': 'Retailers - other', 'count': 8354},
 {'_id': 'Mobile caterer', 'count': 952},
 {'_id': 'Manufacturers/packers', 'count': 437}]


In [27]:
# add restaurant called penang flavours
# didn't need to know the naming convention, copy-paste from canvas
# removing _id entirely, such that mongo generates a new one
estab_addition = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [28]:
# insert new restaurant into the collection
establishments.insert_one(estab_addition)

In [29]:
# check that the new restaurant is there, it also has a brand new _id, yay
establishments.find_one({'BusinessName': 'Penang Flavours'})

{'_id': ObjectId('65055dc9971c31fd40d26db1'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

In [30]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessTypeID', 'BusinessType'}
results = establishments.find_one(query,fields)
pprint(results)

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('650547dd971c31fd40d1d24d')}


In [31]:
# updating halal restaurant with it's proper BusinessTypeID
establishments.update_one({'BusinessName': 'Penang Flavours'}, {'$set': {'BusinessTypeID': 1}})

In [32]:
# confirm update
pprint(establishments.find_one({'BusinessName':'Penang Flavours'}))

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('65055dc9971c31fd40d26db1'),
 'geocode': {'latitude': '51.49014200', 'longitude': '0.08384000'},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


In [33]:
# The magazine is not interested in any establishments in Dover, so check 
# how many documents contain the Dover Local Authority. Then, remove any
# establishments within the Dover Local Authority from the database, and check
# the number of documents to ensure they were deleted.
localauth_dover = {'LocalAuthorityName': 'Dover'}
print(f"Number of Dover's in result: ", establishments.count_documents(localauth_dover))

Number of Dover's in result:  994


In [36]:
# remove dover documents
remove_dover = establishments.delete_many({'LocalAuthorityName': 'Dover'})
# confirm removal
print(f"Number of Dover's in result: ", establishments.count_documents(localauth_dover))

Number of Dover's in result:  0


In [50]:
# checking that i didn't destroy the db
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessName','LocalAuthorityName', 'AddressLine3'}
results = establishments.find_one(query,fields)
pprint(results)

{'AddressLine3': 'Folkestone',
 'BusinessName': 'The Pavilion',
 'LocalAuthorityName': 'Folkestone and Hythe',
 '_id': ObjectId('650547dd971c31fd40d1d534')}


In [47]:
# setting latitude, longitude numbers to decimals
establishments.update_many({}, [{'$set': {"geocode.latitude": {'$toDecimal': "$geocode.latitude"}}}])
establishments.update_many({}, [{'$set': {"geocode.longitude": {'$toDecimal': "$geocode.longitude"}}}])
# setting RatingValue to integers, setting non 1-5 ratings to null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])
establishments.update_many({}, [{'$set': {"RatingValue": {'$toInt': "$RatingValue"}}}])

In [48]:
# double checking results
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = {'BusinessName','geocode.latitude', 'geocode.longitude', 'RatingValue'}
results = establishments.find_one(query,fields)
pprint(results)

{'BusinessName': 'The Pavilion',
 'RatingValue': 5,
 '_id': ObjectId('650547dd971c31fd40d1d534'),
 'geocode': {'latitude': Decimal128('51.0838120000000'),
             'longitude': Decimal128('1.19562500000000')}}
